# Installing full profile Tanzu Application Platform

In [ ]:
cd $HOME/projects/tap-bounty/install/scripts

In [ ]:
chmod 755 setUp.sh

In [ ]:
./setUp.sh 

In [ ]:
cd $HOME/projects/tap-bounty

Update tap-values.yaml to reflect your own environment

In [ ]:
tanzu package install tap -p tap.tanzu.vmware.com -v 1.0.0 --values-file tap-values.yml -n tap-install

In [ ]:
tanzu package installed get tap -n tap-install

In [ ]:
tanzu package installed list -A

## Configure Ingress and Routing

In [ ]:
kubectl get svc envoy -n tanzu-system-ingress

In [ ]:
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction start --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction add 35.226.28.109 --name="tap-gui.tap.tanzu4u.net." --ttl="300" --type="A" --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction execute --zone="tanzu4u"

In [ ]:
cd tap-ingress

In [ ]:
./configure-ingress.sh values-ingress.yaml

In [ ]:
nslookup tap-gui.tap.tanzu4u.net

In [ ]:
open -a "Google Chrome" https://tap-gui.tap.tanzu4u.net

## Deploy a sample Java application

In [ ]:
tanzu apps workload create tanzu-java-web-app \
--git-repo https://github.com/sreeramsunkara/tanzu-java-web-app \
--git-branch main \
--type web \
--label app.kubernetes.io/part-of=tanzu-java-web-app \
--namespace dev \
--yes

In [ ]:
tanzu apps workload get tanzu-java-web-app -n dev

In [ ]:
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction start --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction add 35.226.28.109 --name="*.apps.tap.tanzu4u.net." --ttl="300" --type="A" --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction execute --zone="tanzu4u"

Postgres Operator installation

In [ ]:
cd ~/projects/tap-bounty/postgres

In [ ]:
chmod 755 prereq.sh

Edit prereq.sh and update username and password for Tanzu Network Registry.

In [ ]:
./prereq.sh

In [ ]:
cat postgres-operator/values.yaml

kubectl create ns postgres-operator

kubectl create secret docker-registry regsecret \
    --docker-server=https://registry.pivotal.io/ \
    --docker-username='TANZU_NETWORK_USERNAME' \
    --docker-password='TANZU_NETWORK_PASSWORD' \
    --namespace=postgres-operator

helm install my-postgres-operator postgres-operator/ \
  --namespace=postgres-operator \
  --wait    

kubectl get serviceaccount -n postgres-operator

kubectl get all --selector app=postgres-operator -n postgres-operator

kubectl logs -l app=postgres-operator -n postgres-operator

kubectl api-resources --api-group=sql.tanzu.vmware.com -n postgres-operator

In [ ]:
./install-operator.sh

In [ ]:
helm list -n postgres-operator

In [ ]:
kubectl get storageclasses

kubectl create secret docker-registry regsecret \
    --docker-server=https://registry.pivotal.io/ \
    --docker-username='TANZU_NETWORK_USERNAME' \
    --docker-password='TANZU_NETWORK_PASSWORD' \
    --namespace=dev

kubectl apply -f postgres.yaml

kubectl get postgres/postgres-db -n dev

kubectl apply -f postgres-service-binding-rbac.yaml

In [ ]:
./install-instance.sh

In [ ]:
kubectl get postgres/postgres-db -n dev

Checkout the project https://github.com/sreeramsunkara/spring-petclinic-tap and cd into the folder.
Ensure spring.datasource.initialization-mode=always to create the schema and data application.properties and application-postgres.properties
For subsequent runs set spring.datasource.initialization-mode=never

In [ ]:
cd ~/projects/tap/spring-petclinic-tap

In [ ]:
tanzu apps workload apply -f kubernetes/tap/workload.yaml -y

In [ ]:
tanzu apps workload get spring-petclinic -n dev